In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [28]:
!ls

datalab    DI_WOF  drive		    ngrok
DIL_LineX  DI_WOS  learning_by_association  ngrok-stable-linux-amd64.zip


In [4]:
!git clone https://github.com/jasondias9/learning_by_association.git

Cloning into 'learning_by_association'...
remote: Counting objects: 439, done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 439 (delta 59), reused 54 (delta 27), pack-reused 352
Receiving objects: 100% (439/439), 395.86 KiB | 9.90 MiB/s, done.
Resolving deltas: 100% (290/290), done.


In [0]:
import os
os.chdir('/content/learning_by_association/')

In [0]:
!cp -r /content/drive/DI_WOF/ /content/DI_WOF

In [0]:
!cp -r /content/drive/DI_WOS/ /content/DI_WOS

In [0]:
!cp -r /content/drive/DIL/ /content/DIL/

In [0]:
import os
os.chdir('learning_by_association/semisup/')

In [23]:
from semisup.tools import nhltb as nl
nl.DATADIR = '/content/DIL_LineX/'
train_images, train_labels, unsup = nl.get_data('train')
test_images, test_labels, _ = nl.get_data('test')
train_images / 255.
test_images / 255.
print('[INFO]   Data imported...')

[INFO]   Reading training images...
[INFO]   Reading testing images...
[INFO]   Data imported...


In [0]:
os.chdir('/content/learning_by_association/')
import semisup
os.chdir('semisup')
import tools

In [32]:
from tools import nhltb as nl
nl.DATADIR = '/content/DIL/'
train_images, train_labels, unsup = nl.get_data('train')
test_images, test_labels, _ = nl.get_data('test')
train_images / 255.
test_images / 255.
print('[INFO]   Data imported...')

[INFO]   Reading training images...
[INFO]   Reading testing images...
[INFO]   Data imported...


(391, 144, 256, 3)

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import sys
import numpy as np
import semisup

from tensorflow.python.platform import app

# Number of labeled samples used per class
SUP_P_CLASS = 40

# Integer random seed used for labeled set selection.
SUP_SEED = 1         

# Number of labeled samples per class per batch
SUP_P_BATCH = 40
                
# Number of unlabeled samples per batch.
UNSUP_BATCH_SIZE = 50

# Number of steps between evaluations
EVAL_INTERVAL = 5
                 
# Initial learning rate
LEARNING_RATE =  1e-4

# Learning rate decay factor.
DECAY_FACTOR = 0.33

# Learning rate decay interval in steps
DECAY_STEPS = 150
                  
#Weight for visit loss.
VISIT_WEIGHT = 0.1

# Number of training steps
MAX_STEPS = 200

# Training log path
LOGDIR = '/tmp/nhltb_DIL_LineX'

NUM_LABELS = nl.NUM_LABELS
IMAGE_SHAPE = nl.IMAGE_SHAPE

import logging
logging.getLogger().setLevel(logging.INFO)

In [34]:



# Sample labeled training subset.
seed = SUP_SEED if SUP_SEED != -1 else None

sup_by_label = semisup.sample_by_label(train_images, train_labels,
                                     SUP_P_CLASS, NUM_LABELS, seed)

graph = tf.Graph()
with graph.as_default():
    model = semisup.SemisupModel(semisup.architectures.nhltb_model, NUM_LABELS,
                                 IMAGE_SHAPE)


    print('[INFO]    Creating sup images...')
    t_sup_images, t_sup_labels  = semisup.create_input(train_images, train_labels,
                                         SUP_P_BATCH)

    print('[INFO]    Creating unsup images...')
    t_unsup_images = semisup.create_input(unsup, None,
                                             UNSUP_BATCH_SIZE)

    # Compute embeddings and logits.
    t_sup_emb = model.image_to_embedding(t_sup_images)
    t_unsup_emb = model.image_to_embedding(t_unsup_images)
    t_sup_logit = model.embedding_to_logit(t_sup_emb)


    # Add losses.
    model.add_semisup_loss(
        t_sup_emb, t_unsup_emb, t_sup_labels, visit_weight=VISIT_WEIGHT)
    model.add_logit_loss(t_sup_logit, t_sup_labels)

    t_learning_rate = tf.train.exponential_decay(
        LEARNING_RATE,
        model.step,
        DECAY_STEPS,
        DECAY_FACTOR,
        staircase=True)
    train_op = model.create_train_op(t_learning_rate)
    summary_op = tf.summary.merge_all()

    summary_writer = tf.summary.FileWriter(LOGDIR, graph)

    saver = tf.train.Saver()


Tensor("net/conv1_1/Elu:0", shape=(?, 144, 256, 32), dtype=float32)
Tensor("net/conv1_2/Elu:0", shape=(?, 144, 256, 32), dtype=float32)
Tensor("net/pool1/MaxPool:0", shape=(?, 72, 128, 32), dtype=float32)
Tensor("net/conv2_1/Elu:0", shape=(?, 72, 128, 64), dtype=float32)
Tensor("net/conv2_2/Elu:0", shape=(?, 72, 128, 64), dtype=float32)
Tensor("net/pool2/MaxPool:0", shape=(?, 36, 64, 64), dtype=float32)
Tensor("net/conv3_1/Elu:0", shape=(?, 36, 64, 128), dtype=float32)
Tensor("net/conv3_2/Elu:0", shape=(?, 36, 64, 128), dtype=float32)
Tensor("net/pool3/MaxPool:0", shape=(?, 18, 32, 128), dtype=float32)
[INFO]    Creating sup images...
(144, 256, 3)
Tensor("input_producer/Gather:0", shape=(144, 256, 3), dtype=uint8)
[INFO]    Creating unsup images...
Tensor("net_2/conv1_1/Elu:0", shape=(40, 144, 256, 32), dtype=float32)
Tensor("net_2/conv1_2/Elu:0", shape=(40, 144, 256, 32), dtype=float32)
Tensor("net_2/pool1/MaxPool:0", shape=(40, 72, 128, 32), dtype=float32)
Tensor("net_2/conv2_1/Elu:

In [0]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    for step in xrange(MAX_STEPS):
      _, summaries = sess.run([train_op, summary_op])
      if (step + 1) % EVAL_INTERVAL == 0 or step == 99:
        print('Step: %d' % step)
        test_pred = model.classify(test_images).argmax(-1)
        train_pred = model.classify(train_images).argmax(-1)
        print(test_pred)
        conf_mtx = semisup.confusion_matrix(test_labels, test_pred, NUM_LABELS)
        test_err = (test_labels != test_pred).mean() * 100
        train_err = (train_labels != train_pred).mean() * 100
        print(conf_mtx)
        print('Test error: %.2f %%' % test_err)
        print('Train error: %.2f %%' % train_err)
        print()

        test_summary = tf.Summary(
            value=[tf.Summary.Value(
                tag='Test Err', simple_value=test_err)])
        train_summary = tf.Summary(
            value=[tf.Summary.Value(
                tag='Train Err', simple_value=train_err)])
        summary_writer.add_summary(summaries, step)
        summary_writer.add_summary(test_summary, step)
        summary_writer.add_summary(train_summary, step)

        saver.save(sess, LOGDIR, model.step)

    coord.request_stop()
    coord.join(threads)
    print('[INFO]    Training complete')

Step: 4
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[[ 0 86]
 [ 0 84]]
Test error: 50.59 %
Train error: 50.13 %

Step: 9
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[[ 0 86]
 [ 0 84]]
Test error: 50.59 %
Train error: 50.13 %

Step: 14
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 

[0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0
 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0
 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]
[[70 16]
 [20 64]]
Test error: 21.18 %
Train error: 19.69 %

Step: 84
[0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0 0 0 1 0
 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0
 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0
 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0]
[[66 20]
 [21 63]]
Test error: 24.12 %
Train error: 18.16 %

Step: 89
[0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0
 1 1 0 1 1 1 1 1 1

[1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0
 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 0 1 0 0
 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0
 0 1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0]
[[75 11]
 [27 57]]
Test error: 22.35 %
Train error: 7.16 %

Step: 159
[1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 0
 0 1 1 1 1 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0]
[[69 17]
 [21 63]]
Test error: 22.35 %
Train error: 5.12 %

Step: 164
[1 1 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0
 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 0
 0 1 0 1 1 1 1 1 0

In [0]:
LOG_DIR = '/tmp/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [14]:
os.chdir('/content')
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2018-04-03 11:45:26--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.243.252.123, 50.17.234.140, 23.23.215.144, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.243.252.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  --.-KB/s    in 0.1s    

2018-04-03 11:45:26 (47.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
os.chdir('/content')
get_ipython().system_raw('./ngrok http 6006 &')

In [16]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://d35f8adc.ngrok.io


In [106]:
!ps -elf | grep ngrok

0 S root     13443     1  0  80   0 - 14293 -      22:24 ?        00:00:06 ./ngrok http 6006
0 S root     16082 22391 99  80   0 -  8491 -      22:52 pts/0    00:00:01 /bin/sh -c ps -elf | grep ngrok
0 S root     16084 16082  0  80   0 -  9550 -      22:52 pts/0    00:00:00 grep ngrok


In [107]:
!ps -elf | grep python

0 S root        90    80  0  80   0 - 45730 -      14:55 ?        00:00:19 /usr/bin/python /usr/local/bin/jupyter-notebook -y --no-browser --log-level=DEBUG --debug --NotebookApp.allow_origin="*" --NotebookApp.log_format="%(message)s" --NotebookApp.token= --Session.key="" --Session.keyfile="" --ContentsManager.untitled_directory="Untitled Folder" --ContentsManager.untitled_file="Untitled File" --ContentsManager.untitled_notebook="Untitled Notebook" --NotebookNotary.algorithm="sha1" --KernelManager.autorestart=True --MultiKernelManager.default_kernel_name="python2" --ip="127.0.0.1" --port=9000 --port-retries=0 --notebook-dir="/content" --NotebookNotary.algorithm=sha256 --NotebookNotary.secret_file=/content/datalab/.config/notary_secret --NotebookApp.base_url=/tun/m/gpu-3b08d9d6-7605-40c1-85a2-3fb71cafbfdb/
0 S root     16085 22391 99  80   0 -  8490 -      22:52 pts/0    00:00:01 /bin/sh -c ps -elf | grep python
0 S root     16087 16085  0  80   0 -  9549 -      22:52 pts/0    00:00:00 

In [0]:
!rm -r /tmp/nhltb_DIL_unsup_500/